In [1]:
import os
import scipy.io as sio
import pickle
import numpy as np

In [38]:
# Function to reorganize and save temporal and spatial gait data
def reorganize_and_save_temporal_spatial_data(input_folder, output_folder):
    """
    Reorganize and save temporal and spatial gait data for participants.
    
    Parameters:
    - input_folder: Path to the folder containing gait data .mat files for each participant.
    - output_folder: Path to the folder where reorganized data will be saved.
    """
    # Create the output folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Iterate over files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.mat'):
            # Load .mat file
            file_path = os.path.join(input_folder, file_name)
            data = sio.loadmat(file_path)
            
            # Extract relevant gait parameters
            try:
                participant_data = {}
                for gait_key in data['Par'].dtype.names:
                    gait_data = data['Par'][gait_key][0, 0]
                    parameters = gait_data['Parameters'][0, 0]
                    
                    # Extract right, left, and combined parameters
                    right_params = parameters['right'][0, 0]
                    left_params = parameters['left'][0, 0]
                    combined_params = parameters['combined'][0, 0]
                    
                    participant_data[gait_key] = {
                        'right_leg': {
                            'cadence': right_params['cadence'].item(),
                            'stride_duration': right_params['strideDuration'].flatten(),
                            'stance_phase': right_params['stancePhase'].flatten(),
                            'swing_phase': right_params['swingPhase'].flatten(),
                            'stride_length': right_params['strideLength'].flatten(),
                            'heel_idx': right_params['heelIdx'].flatten(),
                            'toe_idx': right_params['toeIdx'].flatten()
                        },
                        'left_leg': {
                            'cadence': left_params['cadence'].item(),
                            'stride_duration': left_params['strideDuration'].flatten(),
                            'stance_phase': left_params['stancePhase'].flatten(),
                            'swing_phase': left_params['swingPhase'].flatten(),
                            'stride_length': left_params['strideLength'].flatten(),
                            'heel_idx': left_params['heelIdx'].flatten(),
                            'toe_idx': left_params['toeIdx'].flatten()
                        },
                        'combined': {
                            'cadence': combined_params['cadence'].item(),
                            'stride_duration': combined_params['strideDuration'].flatten(),
                            'stance_phase': combined_params['stancePhase'].flatten(),
                            'swing_phase': combined_params['swingPhase'].flatten(),
                            'right_step_length': combined_params['rightStepLength'].flatten(),
                            'left_step_length': combined_params['leftStepLength'].flatten(),
                            'right_step_time': combined_params['rightStepTime'].flatten(),
                            'left_step_time': combined_params['leftStepTime'].flatten(),
                            'step_length': combined_params['stepLength'].flatten(),
                            'step_duration': combined_params['stepDuration'].flatten(),
                            'right_double_support_time': combined_params['rightDoubleSupportTime'].flatten(),
                            'left_double_support_time': combined_params['leftDoubleSupportTime'].flatten(),
                            'right_single_support_time': combined_params['rightSingleSupportTime'].flatten(),
                            'left_single_support_time': combined_params['leftSingleSupportTime'].flatten(),
                            'r_step_width': combined_params['RstepWidth'].flatten(),
                            'l_step_width': combined_params['LstepWidth'].flatten()
                        },
                        'avg_velocity': parameters['avgVelocity'].item(),
                        'distance_traveled': gait_data['distanceTraveled'].item(),
                        'inst_velocity': gait_data['instVelocity'].flatten()
                    }
                
                # Print data summary before saving for verification
                print(f"Data for {file_name}: {participant_data}")
                
                # Save the reorganized data in the output folder as a .pkl file
                output_file_name = file_name.replace('.mat', '_reorganized.pkl')
                output_file_path = os.path.join(output_folder, output_file_name)
                
                with open(output_file_path, 'wb') as f:
                    pickle.dump(participant_data, f)
                
                # Verification print to check if data is saved correctly
                print(f"Saved data to {output_file_path}")
            except KeyError as e:
                print(f"KeyError: {e} in file {file_name}")


In [40]:
# Function to concatenate temporal and spatial gait data from multiple participants
def concatenate_temporal_spatial_data(input_folder, output_file):
    """
    Concatenate temporal and spatial gait data from multiple participants and save in a single file.
    
    Parameters:
    - input_folder: Path to the folder containing reorganized gait data .pkl files for each participant.
    - output_file: Path to save the concatenated data.
    """
    concatenated_data = {}
    
    # Iterate over files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('_reorganized.pkl'):
            # Load .pkl file
            file_path = os.path.join(input_folder, file_name)
            with open(file_path, 'rb') as f:
                participant_data = pickle.load(f)
                
                # Concatenate data by header
                for header, gait_data in participant_data.items():
                    if header not in concatenated_data:
                        concatenated_data[header] = {
                            'right_leg': {
                                'cadence': [],
                                'stride_duration': [],
                                'stance_phase': [],
                                'swing_phase': [],
                                'stride_length': [],
                                'heel_idx': [],
                                'toe_idx': []
                            },
                            'left_leg': {
                                'cadence': [],
                                'stride_duration': [],
                                'stance_phase': [],
                                'swing_phase': [],
                                'stride_length': [],
                                'heel_idx': [],
                                'toe_idx': []
                            },
                            'combined': {
                                'cadence': [],
                                'stride_duration': [],
                                'stance_phase': [],
                                'swing_phase': [],
                                'right_step_length': [],
                                'left_step_length': [],
                                'right_step_time': [],
                                'left_step_time': [],
                                'step_length': [],
                                'step_duration': [],
                                'right_double_support_time': [],
                                'left_double_support_time': [],
                                'right_single_support_time': [],
                                'left_single_support_time': [],
                                'r_step_width': [],
                                'l_step_width': []
                            },
                            'avg_velocity': [],
                            'distance_traveled': [],
                            'inst_velocity': []
                        }
                    
                    # Append participant data to concatenated data
                    for leg in ['right_leg', 'left_leg', 'combined']:
                        for key, value in gait_data[leg].items():
                            concatenated_data[header][leg][key].extend(value if isinstance(value, list) else [value])
                    concatenated_data[header]['avg_velocity'].append(gait_data['avg_velocity'])
                    concatenated_data[header]['distance_traveled'].append(gait_data['distance_traveled'])
                    concatenated_data[header]['inst_velocity'].extend(gait_data['inst_velocity'])
    
    # Save the concatenated data in the output file
    with open(output_file, 'wb') as f:
        pickle.dump(concatenated_data, f)
    
    # Verification print to check if concatenated data is saved correctly
    print(f"Concatenated data saved to {output_file}")

In [43]:
# Example usage for concatenation
output_file = r'F:\AlterG\TemporoSpatial\Norm\Concatenated_Gait_Data.pkl'
input_folder = r'F:\AlterG\Control\TempSpatial\Gait'
output_folder = r'F:\AlterG\TemporoSpatial\Reorganized'
reorganize_and_save_temporal_spatial_data(input_folder, output_folder)
concatenate_temporal_spatial_data(output_folder, output_file)

Data for GaitParameters01.mat: {'Gait02': {'right_leg': {'cadence': array([[1.31578947]]), 'stride_duration': array([array([[0.95, 0.87, 0.92, 0.9 , 0.92]])], dtype=object), 'stance_phase': array([array([[0.59, 0.52, 0.56, 0.55, 0.59]])], dtype=object), 'swing_phase': array([array([[0.36, 0.35, 0.36, 0.35, 0.33]])], dtype=object), 'stride_length': array([array([[1.21786 , 1.15216 , 1.201613, 1.138127, 1.09293 ]])],
      dtype=object), 'heel_idx': array([array([[  7, 102, 189, 281, 371, 463]], dtype=uint16)],
      dtype=object), 'toe_idx': array([array([[ 66, 154, 245, 336, 430]], dtype=uint16)], dtype=object)}, 'left_leg': {'cadence': array([[1.31578947]]), 'stride_duration': array([array([[0.91, 0.89, 0.91, 0.91, 0.94]])], dtype=object), 'stance_phase': array([array([[0.58, 0.52, 0.59, 0.57, 0.57]])], dtype=object), 'swing_phase': array([array([[0.33, 0.37, 0.32, 0.34, 0.37]])], dtype=object), 'stride_length': array([array([[1.20314 , 1.155242, 1.160457, 1.133871, 1.02917 ]])],
    

In [44]:
##############################################################################################################

In [33]:

def load_and_check_temporal_data(file_path):
    """
    Load and explore saved temporal spatial data to verify correctness.

    Parameters:
    - file_path: Path to the .pkl file to be loaded and checked.
    """
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        return

    # Load the saved pickle file
    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    # Print a summary of the data structure
    print(f"Exploring data in: {file_path}")
    if isinstance(data, dict):
        for header, header_data in data.items():
            print(f"Header: {header}")
            if isinstance(header_data, dict):
                for sub_key, sub_value in header_data.items():
                    if isinstance(sub_value, dict):
                        print(f"  Sub-Key: {sub_key}")
                        for k, v in sub_value.items():
                            print(f"    {k}: Type: {type(v)}, Length: {len(v) if hasattr(v, '__len__') else 'N/A'}")
                    else:
                        print(f"  {sub_key}: Type: {type(sub_value)}, Length: {len(sub_value) if hasattr(sub_value, '__len__') else 'N/A'}")
    else:
        print("Unexpected data structure.")

In [34]:
concatenated_file_path = r'F:\AlterG\TemporoSpatial\ConcatenatedNorm\Concatenated_Gait_Data.pkl'
load_and_check_temporal_data(concatenated_file_path)

Exploring data in: F:\AlterG\TemporoSpatial\ConcatenatedNorm\Concatenated_Gait_Data.pkl
